In [23]:
import torch
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd 
import numpy as np 
# 加载预训练模型和分词器
tokenizer = BertTokenizer.from_pretrained("D:/gitee/bert-base-uncased")
model = BertForMaskedLM.from_pretrained("D:/gitee/bert-base-uncased")

## 用pandas的dataframe格式存储
options = pd.read_table("初中完型填空对应的选项.txt",header=None,sep=",")
options.columns = ["A","B","C","D","RT"]
with open("初中完型填空-Copy1.txt","r") as file:
    tidy_data = file.read()
tidy_data = tidy_data.split("\n")

Some weights of the model checkpoint at D:/gitee/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
words = list(options.iloc[0,:4])
print(words)

['crazy', 'beautiful', 'lazy', 'ugly']


In [11]:
pre_word = list()

for i in range(len(tidy_data)):
    prob = list() #保存当前句子下每种组合的可能性,选出最大的一组
    sentence = tidy_data[i]
    inputs = tokenizer.encode_plus(sentence, return_tensors="pt", add_special_tokens=True)

    input_ids = inputs["input_ids"]
    ## 记录掩码的位置
    mask_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
    
    words = list(options.iloc[i,:4])
    for word in words:
        # 构建待预测的句子
        masked_sentence = sentence.replace("[MASK]", word)
        
        # 使用分词器对句子进行编码
        input_ids = tokenizer.encode(masked_sentence, add_special_tokens=True, return_tensors="pt")
        
        # 使用模型进行推理
        outputs = model(input_ids)
        predictions = outputs.logits
        
        # 获取预测结果对应的概率值
        predicted_token_id = tokenizer.convert_tokens_to_ids(word)
        predicted_probability = predictions[0, mask_index, predicted_token_id].item()
        
        # 打印单词和对应的概率值
        print(f"Word: {word}, Probability: {predicted_probability}")
        prob.append(predicted_probability)
    pre_word.append(options.iloc[i,np.argmax(prob)])
    

Word: crazy, Probability: 15.384073257446289
Word: beautiful, Probability: 14.479395866394043
Word: lazy, Probability: 13.185140609741211
Word: ugly, Probability: 14.475358963012695
Word: think, Probability: 19.187870025634766
Word: hope, Probability: 16.129600524902344
Word: suggest, Probability: 12.45032787322998
Word: ask, Probability: 18.128929138183594
Word: wrong, Probability: 19.49198341369629
Word: welcome, Probability: 8.906497955322266
Word: important, Probability: 13.835725784301758
Word: interesting, Probability: 13.532438278198242
Word: lessons, Probability: 17.660797119140625
Word: symbols, Probability: 9.39571475982666
Word: novels, Probability: 12.453939437866211
Word: results, Probability: 18.832393646240234
Word: stupid, Probability: 15.29840087890625
Word: interested, Probability: 17.857742309570312
Word: sad, Probability: 20.25552749633789
Word: glad, Probability: 14.4566011428833
Word: although, Probability: 16.11528205871582
Word: if, Probability: 18.0899925231933

In [1]:
right_word = [options.loc[i,options.iloc[i,4]] for i in range(10)]

NameError: name 'options' is not defined

预测结果与正确选项对比

In [22]:
[pre_word[i] + " --- " + right_word[i] for i in range(10)]

['crazy --- beautiful',
 'think --- think',
 'wrong --- wrong',
 'results --- results',
 'sad --- sad',
 'before --- if',
 'catches --- likes',
 'early --- together',
 'better --- better',
 'friendship --- friendship']